In [ ]:
!git clone https://github.com/gitlabspy/stylegan2-tf-2.x.git
!gdown --id 1T2GwCxz_vAZY2us4xnljZyIgosfSN9rn
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!rm -r /content/stylegan2-tf-2.x/official-converted/ 
!unzip  official-converted.zip 
!rm -r __MACOSX/ 
!mv /content/official-converted/ /content/stylegan2-tf-2.x/
!mkdir /content/stylegan2-tf-2.x/raw_images
!mkdir /content/stylegan2-tf-2.x/aligned
!mv /content/shape_predictor_68_face_landmarks.dat.bz2 /content/stylegan2-tf-2.x/
%cd /content/stylegan2-tf-2.x/
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
import numpy as np
import tensorflow as tf
import tqdm
import os
from PIL import Image
from stylegan2.utils import postprocess_images
from load_models import load_generator
from copy_official_weights import convert_official_weights_together
from tf_utils import allow_memory_growth

from tensorflow.compat.v1 import ConfigProto, InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
sess = InteractiveSession(config=config)

import matplotlib.pyplot as plt

ckpt_dir_cuda = os.path.join('./official-converted', 'cuda')
g_clone = load_generator(g_params=None, is_g_clone=True, ckpt_dir=ckpt_dir_cuda, custom_cuda=True)
g_clone.trainable = False
from stylegan2.utils import postprocess_images
from stylegan2.utils import adjust_dynamic_range

In [ ]:
import scipy.ndimage
import dlib

#######################
#                     #
#                     #
#   face alignment    #
#                     #
#                     #
#######################
"""
 ref:https://github.com/pbaylies/stylegan-encoder
"""

class LandmarksDetector:
    def __init__(self, predictor_model_path):
        """
        :param predictor_model_path: path to shape_predictor_68_face_landmarks.dat file
        """
        self.detector = dlib.get_frontal_face_detector() # cnn_face_detection_model_v1 also can be used
        self.shape_predictor = dlib.shape_predictor(predictor_model_path)

    def get_landmarks(self, image):
        img = dlib.load_rgb_image(image)
        dets = self.detector(img, 1)

        for detection in dets:
            try:
                face_landmarks = [(item.x, item.y) for item in self.shape_predictor(img, detection).parts()]
                yield face_landmarks
            except:
                print("Exception in get_landmarks()!")

def image_align(src_file, dst_file, face_landmarks, output_size=1024, transform_size=4096, enable_padding=True, x_scale=1, y_scale=1, em_scale=0.1, alpha=False):
        # Align function from FFHQ dataset pre-processing step
        # https://github.com/NVlabs/ffhq-dataset/blob/master/download_ffhq.py

        lm = np.array(face_landmarks)
        lm_chin          = lm[0  : 17]  # left-right
        lm_eyebrow_left  = lm[17 : 22]  # left-right
        lm_eyebrow_right = lm[22 : 27]  # left-right
        lm_nose          = lm[27 : 31]  # top-down
        lm_nostrils      = lm[31 : 36]  # top-down
        lm_eye_left      = lm[36 : 42]  # left-clockwise
        lm_eye_right     = lm[42 : 48]  # left-clockwise
        lm_mouth_outer   = lm[48 : 60]  # left-clockwise
        lm_mouth_inner   = lm[60 : 68]  # left-clockwise

        # Calculate auxiliary vectors.
        eye_left     = np.mean(lm_eye_left, axis=0)
        eye_right    = np.mean(lm_eye_right, axis=0)
        eye_avg      = (eye_left + eye_right) * 0.5
        eye_to_eye   = eye_right - eye_left
        mouth_left   = lm_mouth_outer[0]
        mouth_right  = lm_mouth_outer[6]
        mouth_avg    = (mouth_left + mouth_right) * 0.5
        eye_to_mouth = mouth_avg - eye_avg

        # Choose oriented crop rectangle.
        x = eye_to_eye - np.flipud(eye_to_mouth) * [-1, 1]
        x /= np.hypot(*x)
        x *= max(np.hypot(*eye_to_eye) * 2.0, np.hypot(*eye_to_mouth) * 1.8)
        x *= x_scale
        y = np.flipud(x) * [-y_scale, y_scale]
        c = eye_avg + eye_to_mouth * em_scale
        quad = np.stack([c - x - y, c - x + y, c + x + y, c + x - y])
        qsize = np.hypot(*x) * 2

        # Load in-the-wild image.
        if not os.path.isfile(src_file):
            print('\nCannot find source image.')
            return
        img = Image.open(src_file).convert('RGBA').convert('RGB')

        # Shrink.
        shrink = int(np.floor(qsize / output_size * 0.5))
        if shrink > 1:
            rsize = (int(np.rint(float(img.size[0]) / shrink)), int(np.rint(float(img.size[1]) / shrink)))
            img = img.resize(rsize, Image.ANTIALIAS)
            quad /= shrink
            qsize /= shrink

        # Crop.
        border = max(int(np.rint(qsize * 0.1)), 3)
        crop = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
        crop = (max(crop[0] - border, 0), max(crop[1] - border, 0), min(crop[2] + border, img.size[0]), min(crop[3] + border, img.size[1]))
        if crop[2] - crop[0] < img.size[0] or crop[3] - crop[1] < img.size[1]:
            img = img.crop(crop)
            quad -= crop[0:2]

        # Pad.
        pad = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
        pad = (max(-pad[0] + border, 0), max(-pad[1] + border, 0), max(pad[2] - img.size[0] + border, 0), max(pad[3] - img.size[1] + border, 0))
        if enable_padding and max(pad) > border - 4:
            pad = np.maximum(pad, int(np.rint(qsize * 0.3)))
            img = np.pad(np.float32(img), ((pad[1], pad[3]), (pad[0], pad[2]), (0, 0)), 'reflect')
            h, w, _ = img.shape
            y, x, _ = np.ogrid[:h, :w, :1]
            mask = np.maximum(1.0 - np.minimum(np.float32(x) / pad[0], np.float32(w-1-x) / pad[2]), 1.0 - np.minimum(np.float32(y) / pad[1], np.float32(h-1-y) / pad[3]))
            blur = qsize * 0.02
            img += (scipy.ndimage.gaussian_filter(img, [blur, blur, 0]) - img) * np.clip(mask * 3.0 + 1.0, 0.0, 1.0)
            img += (np.median(img, axis=(0,1)) - img) * np.clip(mask, 0.0, 1.0)
            img = np.uint8(np.clip(np.rint(img), 0, 255))
            if alpha:
                mask = 1-np.clip(3.0 * mask, 0.0, 1.0)
                mask = np.uint8(np.clip(np.rint(mask*255), 0, 255))
                img = np.concatenate((img, mask), axis=2)
                img = Image.fromarray(img, 'RGBA')
            else:
                img = Image.fromarray(img, 'RGB')
            quad += pad[:2]

        # Transform.
        img = img.transform((transform_size, transform_size), Image.QUAD, (quad + 0.5).flatten(), Image.BILINEAR)
        if output_size < transform_size:
            img = img.resize((output_size, output_size), Image.ANTIALIAS)

        # Save aligned image.
        img.save(dst_file, 'PNG')


def align_folder(RAW_IMAGES_DIR='./raw_images/', ALIGNED_IMAGES_DIR='./aligned/'):
    landmarks_detector = LandmarksDetector('./shape_predictor_68_face_landmarks.dat')
    for img_name in os.listdir(RAW_IMAGES_DIR):
            try:
                raw_img_path = os.path.join(RAW_IMAGES_DIR, img_name)
                fn = face_img_name = '%s_%02d.png' % (os.path.splitext(img_name)[0], 1)
                if os.path.isfile(fn):
                    continue
                for i, face_landmarks in enumerate(landmarks_detector.get_landmarks(raw_img_path), start=1):
                    try:
                        face_img_name = '%s_%02d.png' % (os.path.splitext(img_name)[0], i)
                        aligned_face_path = os.path.join(ALIGNED_IMAGES_DIR, face_img_name)
                        image_align(raw_img_path, aligned_face_path, face_landmarks, output_size=1024, x_scale=1, y_scale=1)
                    except:
                        print("Exception in face alignment!")
            except:
                print("Exception in landmark detection!")


In [ ]:
from google.colab import files
import shutil
uploaded = files.upload() 
for k,v in uploaded.items():
    shutil.move('./{}'.format(k), './raw_images/{}'.format(k))
align_folder()

In [ ]:
#######################
#                     #
#                     #
#    GAN Inversion    #
#                     #
#                     #
#######################
"""
    ref:image2stylegan https://arxiv.org/abs/1904.03189
"""
def postprocess_images(images):
    # stylegan out
    images = adjust_dynamic_range(images, range_in=(-1.0, 1.0), range_out=(0.0, 255.0), out_dtype=tf.dtypes.float32)
    images = tf.transpose(images, [0, 2, 3, 1])
    return images
def genenrate_from_w(w):
    # 1, 18, 512 -> 1, 1024, 1024, 3
    return postprocess_images(g_clone.generate_from_w_from_wspace(w))
def preposccess(x, img_shape=256):
    # 1024 -> img_shape
    return tf.keras.applications.vgg16.preprocess_input(tf.image.resize(x, (img_shape, img_shape)))
def VGG_model(img_shape=256):
    # p loss
    input_tensor = tf.keras.Input(shape=(img_shape, img_shape, 3))
    VGG = tf.keras.applications.VGG16(include_top=False, weights=None, input_tensor=input_tensor)
    VGGTF = tf.keras.applications.VGG16(include_top=False, weights='imagenet')
    for new_layer, layer in zip(VGG.layers[1:], VGGTF.layers[1:]):
        new_layer.set_weights(layer.get_weights())
    out1 = tf.keras.layers.Flatten()(VGG.get_layer("block1_conv1").output)
    out2 = tf.keras.layers.Flatten()(VGG.get_layer("block1_conv2").output)
    out3 = tf.keras.layers.Flatten()(VGG.get_layer("block3_conv2").output)
    out4 = tf.keras.layers.Flatten()(VGG.get_layer("block4_conv2").output)
    return tf.keras.Model(input_tensor,[out1, out2, out3, out4])
MSE = tf.keras.losses.MeanSquaredError()

@tf.function
def projection_step(x, w, model, optimizer):
    with tf.GradientTape() as tape:
        tape.watch(w)
        img = genenrate_from_w(w)
        img = preposccess(img)
        x = preposccess(x)
        l2loss = MSE(x, img) 
        out1, out2, out3, out4 = model(x)
        pout1, pout2, pout3, pout4 = model(img)
        ploss = MSE(out1 , pout1) + MSE(out2 , pout2) + MSE(out2 , pout2) + MSE(out2 , pout2)
        total_loss = l2loss  + ploss 
    grads = tape.gradient(total_loss, (w, ))
    optimizer.apply_gradients(zip(grads, (w, )))
    return tf.reduce_mean(total_loss)

def projection(x, opt=tf.keras.optimizers.Adam(1e-2, epsilon=1e-08), iters=5000, print_fq=1000):
    w = tf.repeat(g_clone.layers[0]([tf.random.normal((1, 512)), None])[:,tf.newaxis,...], 18, axis=1)
    w = tf.Variable(w, trainable=True)
    vgg = VGG_model()
    for i in range(iters):
        loss = projection_step(x, w, vgg, opt)
        if i % print_fq == 0:
            print("iter : {}/{},  loss : {} ".format(i, iters, loss.numpy()))
    return tf.identity(w.value())

w = projection(np.array(Image.open('./aligned/biden_01.png'))[np.newaxis,...].astype(np.float32))